# Generative Adversarial Network

In this notebook, we'll be building a generative adversarial network (GAN) trained on the network flow dataset.

## Load and normalize data

In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from importlib import reload
import os
import utils, models
reload(utils)
reload(models)
from utils import model_inputs, get_flow_dataset
from models import generator, discriminator
from sklearn.preprocessing import normalize

In [3]:
(benign_train, benign_test), (attack_train, attack_test) = get_flow_dataset(os.path.join('data', 'ids2017_sampled.csv'))

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
benign_train.shape

(1500, 41)

In [5]:
benign_test.shape

(3500, 41)

In [6]:
attack_train.shape

(1650, 41)

In [7]:
attack_test.shape

(3850, 41)

In [8]:
def max_norm(dataset):
    dataset = dataset - dataset.min(axis=0)
    dataset = normalize(dataset, axis=0, norm='max')
    return dataset

In [9]:
# normalize data
benign_train = max_norm(benign_train)
benign_test = max_norm(benign_test)
attack_train = max_norm(attack_train)
attack_test = max_norm(attack_test)

## Hyperparameters

In [10]:
# Size of input flow to discriminator
input_size = benign_train.shape[1]
# Size of latent vector to generator
z_size = 2
# Sizes of hidden layers in generator and discriminator
g_hidden_size = [128, 64]
d_hidden_size = [128, 64]
# Leak factor for leaky ReLU
alpha = 0.01

## Build network

Now we're building the network from the functions defined above.

First is to get our inputs, `input_benign, input_z, input_attack_remains` from `model_inputs` using the sizes of the input and z.

Then, we'll create the generator, `generator(input_z, z_size)`. This builds the generator with the appropriate input and output sizes.

Then the discriminators. We'll build two of them, one for benign flow data and one for attack flow data. Since we want the weights to be the same for both benign and attack flow data, we need to reuse the variables. For the attack flow data, we're getting it from the output of the generator concatenated with remaining part of attack feature called `g_model`. So the benign data discriminator is `discriminator(input_benign)` while the attack discriminator is `discriminator(g_model, reuse=True)`.

In [11]:
tf.reset_default_graph()

# Create our input placeholders
input_benign, input_z, input_attack_remains = model_inputs(input_size, z_size, attack_remains_dim=input_size - z_size)

# Build the model
z_generated, g_hidden = generator(input_z, z_size, n_units=[z_size] + g_hidden_size, alpha=alpha)
g_model = tf.concat([z_generated, input_attack_remains], 1)
# g_model is the generator output concatenated with the remaining part of attack features

d_model_benign, d_logits_benign, d_hidden_benign = discriminator(input_benign,
                                                                 n_units= [input_size] + d_hidden_size,
                                                                 alpha=alpha)
d_model_attack, d_logits_attack, d_hidden_attack = discriminator(g_model,
                                                                 reuse=True,
                                                                 n_units=[input_size] + d_hidden_size,
                                                                 alpha=alpha)

## Discriminator and Generator Losses

Now we need to calculate the losses, which is a little tricky. For the discriminator, the total loss is the sum of the losses for benign and attack flows, `d_loss = d_loss_benign + d_loss_attack`. The losses will by sigmoid cross-entropys, which we can get with `tf.nn.sigmoid_cross_entropy_with_logits`. We'll also wrap that in `tf.reduce_mean` to get the mean for all the flows in the batch. So the losses will look something like 

```python
tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
```

For the benign flow logits, we'll use `d_logits_benign` which we got from the discriminator in the cell above. For the labels, we want them to be all zeros, since these are all benign flows. In TensorFlow, it looks something like `labels = tf.zeros_like(tensor)`

The discriminator loss for the attack flow data is similar. The logits are `d_logits_attack`, which we got from passing the generator output concatenated with remaing part of attack flow features to the discriminator. These attack logits are used with labels of all ones. Remember that we want the discriminator to output 0 for benign flows and 1 for attack flows, so we need to set up the losses to reflect that.

Finally, the generator losses are using `d_logits_attack`, the attack flow logits. But, now the labels are all zeros. The generator is trying to fool the discriminator, so it wants to discriminator to output zeros for attack flows.

In [12]:
# Calculate losses
d_loss_benign = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_benign, 
                                                          labels=tf.zeros_like(d_logits_benign)))
d_loss_attack = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_attack, 
                                                          labels=tf.ones_like(d_logits_attack)))
d_loss = d_loss_benign + d_loss_attack

g_loss = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_attack,
                                                     labels=tf.zeros_like(d_logits_attack)))

## Optimizers

We want to update the generator and discriminator variables separately. So we need to get the variables for each part build optimizers for the two parts. To get all the trainable variables, we use `tf.trainable_variables()`. This creates a list of all the variables we've defined in our graph.

For the generator optimizer, we only want to generator variables. Our past selves were nice and used a variable scope to start all of our generator variable names with `generator`. So, we just need to iterate through the list from `tf.trainable_variables()` and keep variables to start with `generator`. Each variable object has an attribute `name` which holds the name of the variable as a string (`var.name == 'weights_0'` for instance). 

We can do something similar with the discriminator. All the variables in the discriminator start with `discriminator`.

Then, in the optimizer we pass the variable lists to `var_list` in the `minimize` method. This tells the optimizer to only update the listed variables. Something like `tf.train.AdamOptimizer().minimize(loss, var_list=var_list)` will only train the variables in `var_list`.

In [13]:
# Optimizers
learning_rate = 0.002

# Get the trainable_variables, split into G and D parts
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

## Training

In [ ]:
from copy import deepcopy
import time

batch_size = 100
epochs = 1000
losses = []
original_flows, generated_flows, attack_scores = [], [], []
# Only save generator variables
saver = tf.train.Saver(var_list=g_vars)
# TODO: add outer-loop, which is test number, to see convergence trends
# TODO: modify training process - choose random 2 features and give 2 outputs randomly back
with tf.Session() as sess:
    total_start = time.time()
    sess.run(tf.global_variables_initializer())
    init_benign_test_score = sess.run(d_model_benign, feed_dict={input_benign: benign_test})
    init_attack_test_score = sess.run(d_model_benign, feed_dict={input_benign: attack_test})
    for e in range(epochs):
        start = time.time()
        np.random.shuffle(benign_train)
        np.random.shuffle(attack_train)
        original_flows.append([])
        generated_flows.append([])
        attack_scores.append([])
        for ii in range(benign_train.shape[0]//batch_size):
            batch_benign = benign_train[ii * batch_size:(ii + 1) * batch_size]
            
            batch_attack = attack_train[ii * batch_size:(ii + 1) * batch_size]
            batch_z = batch_attack[:,:z_size]
            batch_attack_remains = batch_attack[:,z_size:]
            
            if ii % 5 == 0:
                original_flow = batch_attack
                generated_flow = sess.run(g_model, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
                attack_score = sess.run(d_model_attack, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
                original_flows[-1].append(deepcopy(original_flow[0]))
                generated_flows[-1].append(generated_flow[0])
                attack_scores[-1].append(attack_score[0][0])
                
            
            # Run optimizers
            _ = sess.run(d_train_opt, feed_dict={input_benign: batch_benign, input_z: batch_z, input_attack_remains: batch_attack_remains})
            _ = sess.run(g_train_opt, feed_dict={input_z: batch_z, input_attack_remains: batch_attack_remains})
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_benign: batch_benign, input_attack_remains: batch_attack_remains})
        train_loss_g = g_loss.eval({input_z: batch_z, input_attack_remains: batch_attack_remains})
            
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g),
              "Time elapsed: {:.4f}".format(time.time() - start)
             )    
        # Save losses to view after training
        losses.append((train_loss_d, train_loss_g))
        
        saver.save(sess, './checkpoints/generator.ckpt')
    print('Total training time:', time.time() - total_start)
    
    final_benign_test_score = sess.run(d_model_benign, feed_dict={input_benign: benign_test})
    final_attack_test_score = sess.run(d_model_benign, feed_dict={input_benign: attack_test})

Epoch 1/1000... Discriminator Loss: 0.6726... Generator Loss: 1.6895 Time elapsed: 0.5439
Epoch 2/1000... Discriminator Loss: 0.2376... Generator Loss: 3.7002 Time elapsed: 0.1246
Epoch 3/1000... Discriminator Loss: 0.1313... Generator Loss: 5.3854 Time elapsed: 0.1185
Epoch 4/1000... Discriminator Loss: 0.0693... Generator Loss: 7.0770 Time elapsed: 0.1205
Epoch 5/1000... Discriminator Loss: 0.0226... Generator Loss: 7.6820 Time elapsed: 0.1226
Epoch 6/1000... Discriminator Loss: 0.0355... Generator Loss: 8.6149 Time elapsed: 0.1201
Epoch 7/1000... Discriminator Loss: 0.1973... Generator Loss: 8.2098 Time elapsed: 0.1309
Epoch 8/1000... Discriminator Loss: 0.0850... Generator Loss: 8.7743 Time elapsed: 0.1174
Epoch 9/1000... Discriminator Loss: 0.0282... Generator Loss: 8.6042 Time elapsed: 0.1226
Epoch 10/1000... Discriminator Loss: 0.0075... Generator Loss: 10.0762 Time elapsed: 0.1278
Epoch 11/1000... Discriminator Loss: 0.0083... Generator Loss: 10.3536 Time elapsed: 0.1243
Epoch 

Epoch 92/1000... Discriminator Loss: 0.0080... Generator Loss: 17.7364 Time elapsed: 0.1212
Epoch 93/1000... Discriminator Loss: 0.0049... Generator Loss: 18.5274 Time elapsed: 0.1203
Epoch 94/1000... Discriminator Loss: 0.0023... Generator Loss: 18.2939 Time elapsed: 0.1207
Epoch 95/1000... Discriminator Loss: 0.0028... Generator Loss: 18.2408 Time elapsed: 0.1246
Epoch 96/1000... Discriminator Loss: 0.0036... Generator Loss: 19.2766 Time elapsed: 0.1174
Epoch 97/1000... Discriminator Loss: 0.0074... Generator Loss: 19.1274 Time elapsed: 0.1267
Epoch 98/1000... Discriminator Loss: 0.0023... Generator Loss: 20.0529 Time elapsed: 0.1273
Epoch 99/1000... Discriminator Loss: 0.0017... Generator Loss: 19.4858 Time elapsed: 0.1303
Epoch 100/1000... Discriminator Loss: 0.0005... Generator Loss: 20.0886 Time elapsed: 0.1291
Epoch 101/1000... Discriminator Loss: 0.0031... Generator Loss: 19.6627 Time elapsed: 0.1223
Epoch 102/1000... Discriminator Loss: 0.0014... Generator Loss: 19.8321 Time e

Epoch 182/1000... Discriminator Loss: 0.0001... Generator Loss: 30.7695 Time elapsed: 0.1336
Epoch 183/1000... Discriminator Loss: 0.0006... Generator Loss: 28.4854 Time elapsed: 0.1219
Epoch 184/1000... Discriminator Loss: 0.0009... Generator Loss: 31.0647 Time elapsed: 0.1216
Epoch 185/1000... Discriminator Loss: 0.0003... Generator Loss: 27.5201 Time elapsed: 0.1207
Epoch 186/1000... Discriminator Loss: 0.0008... Generator Loss: 29.3025 Time elapsed: 0.1217
Epoch 187/1000... Discriminator Loss: 0.0002... Generator Loss: 30.8872 Time elapsed: 0.1284
Epoch 188/1000... Discriminator Loss: 0.0004... Generator Loss: 28.7089 Time elapsed: 0.1190
Epoch 189/1000... Discriminator Loss: 0.0006... Generator Loss: 30.0013 Time elapsed: 0.1192
Epoch 190/1000... Discriminator Loss: 0.0000... Generator Loss: 31.3953 Time elapsed: 0.1209
Epoch 191/1000... Discriminator Loss: 0.0014... Generator Loss: 29.1755 Time elapsed: 0.1187
Epoch 192/1000... Discriminator Loss: 0.0003... Generator Loss: 29.023

Epoch 272/1000... Discriminator Loss: 0.0000... Generator Loss: 35.4712 Time elapsed: 0.1159
Epoch 273/1000... Discriminator Loss: 0.0000... Generator Loss: 37.1038 Time elapsed: 0.1281
Epoch 274/1000... Discriminator Loss: 0.0000... Generator Loss: 34.8738 Time elapsed: 0.1189
Epoch 275/1000... Discriminator Loss: 0.0000... Generator Loss: 39.4358 Time elapsed: 0.1152
Epoch 276/1000... Discriminator Loss: 0.0000... Generator Loss: 34.3128 Time elapsed: 0.1148
Epoch 277/1000... Discriminator Loss: 0.0001... Generator Loss: 37.9851 Time elapsed: 0.1164
Epoch 278/1000... Discriminator Loss: 0.0001... Generator Loss: 36.7749 Time elapsed: 0.1243
Epoch 279/1000... Discriminator Loss: 0.0002... Generator Loss: 36.1211 Time elapsed: 0.1218
Epoch 280/1000... Discriminator Loss: 0.0000... Generator Loss: 36.0458 Time elapsed: 0.1165
Epoch 281/1000... Discriminator Loss: 0.0001... Generator Loss: 36.5680 Time elapsed: 0.1198
Epoch 282/1000... Discriminator Loss: 0.0004... Generator Loss: 36.718

Epoch 361/1000... Discriminator Loss: 0.0002... Generator Loss: 40.4195 Time elapsed: 0.1119
Epoch 362/1000... Discriminator Loss: 0.0000... Generator Loss: 41.6780 Time elapsed: 0.1183
Epoch 363/1000... Discriminator Loss: 0.0000... Generator Loss: 42.2490 Time elapsed: 0.1222
Epoch 364/1000... Discriminator Loss: 0.0000... Generator Loss: 40.9791 Time elapsed: 0.1179
Epoch 365/1000... Discriminator Loss: 0.0000... Generator Loss: 44.0206 Time elapsed: 0.1143
Epoch 366/1000... Discriminator Loss: 0.0000... Generator Loss: 40.1183 Time elapsed: 0.1114
Epoch 367/1000... Discriminator Loss: 0.0000... Generator Loss: 42.0033 Time elapsed: 0.1215
Epoch 368/1000... Discriminator Loss: 0.0001... Generator Loss: 42.6980 Time elapsed: 0.1149
Epoch 369/1000... Discriminator Loss: 0.0000... Generator Loss: 41.5925 Time elapsed: 0.1139
Epoch 370/1000... Discriminator Loss: 0.0000... Generator Loss: 37.8626 Time elapsed: 0.1240
Epoch 371/1000... Discriminator Loss: 0.0000... Generator Loss: 41.740

...

Epoch 541/1000... Discriminator Loss: 0.0001... Generator Loss: 31.8501 Time elapsed: 0.1194
Epoch 542/1000... Discriminator Loss: 0.0001... Generator Loss: 26.2576 Time elapsed: 0.1160
Epoch 543/1000... Discriminator Loss: 0.0001... Generator Loss: 31.4320 Time elapsed: 0.1209
Epoch 544/1000... Discriminator Loss: 0.0000... Generator Loss: 30.1082 Time elapsed: 0.1200
Epoch 545/1000... Discriminator Loss: 0.0000... Generator Loss: 28.9444 Time elapsed: 0.1294
Epoch 546/1000... Discriminator Loss: 0.0000... Generator Loss: 32.0352 Time elapsed: 0.1222
Epoch 547/1000... Discriminator Loss: 0.0000... Generator Loss: 30.1617 Time elapsed: 0.1268
Epoch 548/1000... Discriminator Loss: 0.0001... Generator Loss: 28.5923 Time elapsed: 0.1223
Epoch 549/1000... Discriminator Loss: 0.0001... Generator Loss: 29.2824 Time elapsed: 0.1276
Epoch 550/1000... Discriminator Loss: 0.0001... Generator Loss: 31.5933 Time elapsed: 0.1271
Epoch 551/1000... Discriminator Loss: 0.0001... Generator Loss: 31.177

Epoch 630/1000... Discriminator Loss: 0.0001... Generator Loss: 33.3467 Time elapsed: 0.1224
Epoch 631/1000... Discriminator Loss: 0.0001... Generator Loss: 33.6048 Time elapsed: 0.1160
Epoch 632/1000... Discriminator Loss: 0.0003... Generator Loss: 31.3243 Time elapsed: 0.1224
Epoch 633/1000... Discriminator Loss: 0.0000... Generator Loss: 33.5398 Time elapsed: 0.1224
Epoch 634/1000... Discriminator Loss: 0.0000... Generator Loss: 34.7064 Time elapsed: 0.1223
Epoch 635/1000... Discriminator Loss: 0.0001... Generator Loss: 35.0720 Time elapsed: 0.1264
Epoch 636/1000... Discriminator Loss: 0.0001... Generator Loss: 32.6531 Time elapsed: 0.1153
Epoch 637/1000... Discriminator Loss: 0.0001... Generator Loss: 31.7929 Time elapsed: 0.1241
Epoch 638/1000... Discriminator Loss: 0.0001... Generator Loss: 29.7733 Time elapsed: 0.1151
Epoch 639/1000... Discriminator Loss: 0.0001... Generator Loss: 30.3186 Time elapsed: 0.1173
Epoch 640/1000... Discriminator Loss: 0.0000... Generator Loss: 31.796

Epoch 719/1000... Discriminator Loss: 0.0000... Generator Loss: 32.9586 Time elapsed: 0.1220
Epoch 720/1000... Discriminator Loss: 0.0000... Generator Loss: 34.9033 Time elapsed: 0.1280
Epoch 721/1000... Discriminator Loss: 0.0000... Generator Loss: 32.5509 Time elapsed: 0.1242
Epoch 722/1000... Discriminator Loss: 0.0000... Generator Loss: 34.9276 Time elapsed: 0.1175
Epoch 723/1000... Discriminator Loss: 0.0000... Generator Loss: 36.2633 Time elapsed: 0.1236
Epoch 724/1000... Discriminator Loss: 0.0000... Generator Loss: 33.0934 Time elapsed: 0.1223
Epoch 725/1000... Discriminator Loss: 0.0001... Generator Loss: 33.0686 Time elapsed: 0.1310
Epoch 726/1000... Discriminator Loss: 0.0000... Generator Loss: 34.2058 Time elapsed: 0.1121
Epoch 727/1000... Discriminator Loss: 0.0000... Generator Loss: 36.6130 Time elapsed: 0.1121
Epoch 728/1000... Discriminator Loss: 0.0000... Generator Loss: 37.7381 Time elapsed: 0.1129
Epoch 729/1000... Discriminator Loss: 0.0000... Generator Loss: 35.542

Epoch 809/1000... Discriminator Loss: 0.0000... Generator Loss: 38.8460 Time elapsed: 0.1166
Epoch 810/1000... Discriminator Loss: 0.0000... Generator Loss: 38.5558 Time elapsed: 0.1213
Epoch 811/1000... Discriminator Loss: 0.0000... Generator Loss: 34.0975 Time elapsed: 0.1159
Epoch 812/1000... Discriminator Loss: 0.0000... Generator Loss: 34.7598 Time elapsed: 0.1145
Epoch 813/1000... Discriminator Loss: 0.0000... Generator Loss: 37.3102 Time elapsed: 0.1198
Epoch 814/1000... Discriminator Loss: 0.0000... Generator Loss: 41.8209 Time elapsed: 0.1169
Epoch 815/1000... Discriminator Loss: 0.0000... Generator Loss: 37.9534 Time elapsed: 0.1162
Epoch 816/1000... Discriminator Loss: 0.0000... Generator Loss: 37.1488 Time elapsed: 0.1155
Epoch 817/1000... Discriminator Loss: 0.0000... Generator Loss: 33.2158 Time elapsed: 0.1211
Epoch 818/1000... Discriminator Loss: 0.0000... Generator Loss: 38.1803 Time elapsed: 0.1197
Epoch 819/1000... Discriminator Loss: 0.0000... Generator Loss: 37.310

Epoch 898/1000... Discriminator Loss: 0.0000... Generator Loss: 41.5022 Time elapsed: 0.1132
Epoch 899/1000... Discriminator Loss: 0.0000... Generator Loss: 42.9153 Time elapsed: 0.1118
Epoch 900/1000... Discriminator Loss: 0.0000... Generator Loss: 42.8306 Time elapsed: 0.1128
Epoch 901/1000... Discriminator Loss: 0.0000... Generator Loss: 38.3986 Time elapsed: 0.1263
Epoch 902/1000... Discriminator Loss: 0.0000... Generator Loss: 39.2493 Time elapsed: 0.1174
Epoch 903/1000... Discriminator Loss: 0.0000... Generator Loss: 38.9583 Time elapsed: 0.1152
Epoch 904/1000... Discriminator Loss: 0.0000... Generator Loss: 36.3511 Time elapsed: 0.1204
Epoch 905/1000... Discriminator Loss: 0.0000... Generator Loss: 39.6952 Time elapsed: 0.1166
Epoch 906/1000... Discriminator Loss: 0.0000... Generator Loss: 38.1310 Time elapsed: 0.1248
Epoch 907/1000... Discriminator Loss: 0.0000... Generator Loss: 38.4719 Time elapsed: 0.1163
Epoch 908/1000... Discriminator Loss: 0.0000... Generator Loss: 43.491

## Training loss

Here we'll check out the training losses for the generator and discriminator.

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator')
plt.plot(losses.T[1], label='Generator')
plt.title("Training Losses")
plt.legend()

# Test Part

In [ ]:
# TODO: test if discriminator is fooled

In [ ]:
print('Accuracy before training')
print('Discriminator benign test accuracy:', (init_benign_test_score <= 0.5).sum() / benign_test.shape[0])
print('Discriminator attack test accuracy:', (init_attack_test_score > 0.5).sum() / attack_test.shape[0])
print('Discriminator total test accuracy:',
      ((init_benign_test_score <= 0.5).sum() + (init_attack_test_score > 0.5).sum()) / (benign_test.shape[0] + attack_test.shape[0]))

In [ ]:
print('Accuracy after training')
print('Discriminator benign test accuracy:', (final_benign_test_score <= 0.5).sum() / benign_test.shape[0])
print('Discriminator attack test accuracy:', (final_attack_test_score > 0.5).sum() / attack_test.shape[0])
print('Discriminator total test accuracy:',
      ((final_benign_test_score <= 0.5).sum() + (final_attack_test_score > 0.5).sum()) / (benign_test.shape[0] + attack_test.shape[0]))

# Output Visualization

In [ ]:
import math

def split_to_two_nearest_factor(x):
    sqrt_x = int(math.sqrt(x))
    i = sqrt_x
    while x % i != 0:
        i -= 1
    return (i, x // i)
fig_size = split_to_two_nearest_factor(input_size)
import numpy as np
import matplotlib.pyplot as plt

fooled = 0

# fig=plt.figure(figsize=(fig_size[0] * 5, fig_size[1] * 5))
# columns = 2
rows = len(original_flows)
for i in range(rows):
#     original_img = original_flows[i][0]
#     generated_img = generated_flows[i][0]
#     ax_original = fig.add_subplot(rows, columns, i * 2 + 1)
#     ax_original.set_title('Epoch {epoch}'.format(epoch=i + 1))
#     plt.imshow(original_img.reshape(fig_size[0],fig_size[1]), cmap='gray')
#     ax_generated = fig.add_subplot(rows, columns, i * 2 + 2)
#     plt.imshow(generated_img.reshape(fig_size[0],fig_size[1]), cmap='gray')
    if attack_scores[i][0] <= 0.5:
#         ax_generated.set_title('Fooled:' + str(attack_scores[i][0]))
        fooled += 1
#     else:
#         ax_generated.set_title('Just close:' + str(attack_scores[i][0]))
# plt.show()
print('Fooled rate:', fooled / len(original_flows))

# Generate attack flows

In [ ]:
#TODO: generate attack flows for Kaihua's discrimination